In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install stardist csbdeep tensorflow tifffile imagecodecs reportlab

# Clone Real-ESRGAN and enter the Real-ESRGAN directory
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Modify degradations.py to fix the import error
degradations_file_path = '/usr/local/lib/python3.10/dist-packages/basicsr/data/degradations.py'

with open(degradations_file_path, 'r') as file:
    file_data = file.read()

# Replace the incorrect import statement
file_data = file_data.replace('from torchvision.transforms.functional_tensor import rgb_to_grayscale',
                              'from torchvision.transforms.functional import rgb_to_grayscale')

# Write the modified data back to the file
with open(degradations_file_path, 'w') as file:
    file.write(file_data)

import os
import shutil
import glob
import zipfile
import tifffile
import numpy as np
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from skimage.io import imsave, imread
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import exposure
from matplotlib.backends.backend_pdf import PdfPages
from PIL import Image

# Function to check and adjust image contrast
def adjust_contrast(image):
    p2, p98 = np.percentile(image, (2, 98))
    image = exposure.rescale_intensity(image, in_range=(p2, p98))
    return image

# Function to convert TIFF to PNG
def tiff_to_png(image_files, png_folder):
    if os.path.isdir(png_folder):
        shutil.rmtree(png_folder)
    os.mkdir(png_folder)

    png_files = []
    for tiff_file in image_files:
        with tifffile.TiffFile(tiff_file) as tif:
            images = [tif.asarray(series=i) for i in range(len(tif.series))]
            for i, image in enumerate(images):
                if image.ndim == 3 and image.shape[2] == 3:  # Check if the image is RGB
                    png_path = os.path.join(png_folder, f"{os.path.basename(tiff_file).split('.')[0]}_{i}.png")
                    Image.fromarray(image).save(png_path)
                    png_files.append(png_path)
    return png_files

# Function to convert PNG back to TIFF
def png_to_tiff(png_files, tiff_folder):
    if os.path.isdir(tiff_folder):
        shutil.rmtree(tiff_folder)
    os.mkdir(tiff_folder)

    tiff_files = []
    for png_file in png_files:
        image = imread(png_file)
        tiff_path = os.path.join(tiff_folder, f"{os.path.basename(png_file).split('.')[0]}.tif")
        imsave(tiff_path, image)
        tiff_files.append(tiff_path)
    return tiff_files

# Function to upsample images using Real-ESRGAN
def upsample_images(image_files, scale=6):
    %cd /content/Real-ESRGAN
    upload_folder = 'upload'
    result_folder = 'results'
    if os.path.isdir(upload_folder):
        shutil.rmtree(upload_folder)
    if os.path.isdir(result_folder):
        shutil.rmtree(result_folder)
    os.mkdir(upload_folder)
    os.mkdir(result_folder)

    for image_file in image_files:
        shutil.copy(image_file, upload_folder)

    # Run Real-ESRGAN
    !python inference_realesrgan.py -n RealESRGAN_x4plus -i {upload_folder} --outscale {scale}

    upsampled_files = sorted(glob.glob(os.path.join(result_folder, '*')))
    return upsampled_files

# Path to the downloaded data
data_path = '/content/drive/My Drive/livetest/mono-20240525T044549Z-001.zip'
extract_path = '/content/sample_data/'

# Extract the zip file
print("Extracting ZIP file...")
with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction completed.")

# Load pre-trained StarDist model
print("Loading StarDist model...")
model = StarDist2D.from_pretrained('2D_versatile_fluo')
print("Model loaded successfully.")

extract_path = '/content/sample_data/mono/'
output_dir = '/content/output/'
os.makedirs(output_dir, exist_ok=True)

# Collect all RGB images
rgb_image_files = []
image_files = glob.glob(os.path.join(extract_path, '*.tif'))
for image_file in image_files:
    with tifffile.TiffFile(image_file) as tif:
        images = [tif.asarray(series=i) for i in range(len(tif.series))]
        for i, image in enumerate(images):
            if image.ndim == 3 and image.shape[2] == 3:  # Check if the image is RGB
                rgb_image_files.append(image_file)
                break

print(f"Found {len(rgb_image_files)} RGB image files for upsampling.")

# Step 1: Convert original RGB TIF images to grayscale and save as TIF
def convert_rgb_to_grayscale_tif(image_files, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    grayscale_tiff_files = []
    for image_file in image_files:
        with tifffile.TiffFile(image_file) as tif:
            images = [tif.asarray(series=i) for i in range(len(tif.series))]
            for i, image in enumerate(images):
                if image.ndim == 3 and image.shape[2] == 3:  # Check if the image is RGB
                    gray_image = rgb2gray(image)
                    gray_image = (gray_image * 255).astype(np.uint8)
                    tiff_path = os.path.join(output_folder, f"{os.path.basename(image_file).split('.')[0]}_gray_{i}.tif")
                    imsave(tiff_path, gray_image)
                    grayscale_tiff_files.append(tiff_path)
    return grayscale_tiff_files

grayscale_tiff_folder = os.path.join(output_dir, 'grayscale_tiff')
grayscale_tiff_files = convert_rgb_to_grayscale_tif(rgb_image_files, grayscale_tiff_folder)

# Convert TIFF to PNG for upsampling
png_folder = 'png_images'
png_files = tiff_to_png(rgb_image_files, png_folder)

# Upsample the PNG images
upsampled_png_files = upsample_images(png_files)

# Convert the upsampled PNG images back to TIFF
upsampled_tiff_folder = os.path.join(output_dir, 'upsampled_tiff')
upsampled_tiff_files = png_to_tiff(upsampled_png_files, upsampled_tiff_folder)

# Sort all files to ensure the order is maintained
rgb_image_files.sort()
grayscale_tiff_files.sort()
upsampled_tiff_files.sort()

# Function to process images and save figures into a PDF
def process_images(rgb_files, grayscale_files, upsampled_files, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    pdf_path = os.path.join(output_folder, 'report.pdf')
    with PdfPages(pdf_path) as pdf:
        for i in range(len(rgb_files)):
            try:
                # Ensure each set of images are processed in order
                rgb_image_file = rgb_files[i]
                grayscale_image_file = grayscale_files[i]
                upsampled_image_file = upsampled_files[i]

                # Process and save images
                image_files = [rgb_image_file, grayscale_image_file, upsampled_image_file]
                base_name = os.path.basename(rgb_image_file).split('.')[0]  # Use original TIF filename as the base name
                for j, image_file in enumerate(image_files):
                    print(f"Processing file: {image_file}")
                    with tifffile.TiffFile(image_file) as tif:
                        image = tif.asarray(series=0)  # Only process the first series for each file

                        # Print shape and type of image for debugging
                        print(f"Processing image from file {image_file}")
                        print(f"Image shape: {image.shape}, Image dtype: {image.dtype}")

                        # Convert RGB images to grayscale if necessary
                        if image.ndim == 3 and image.shape[2] == 3 and j != 1:  # Only convert for RGB images, not for grayscale
                            image = rgb2gray(image)
                            image = (image * 255).astype(np.uint8)

                        # Adjust image contrast
                        image = adjust_contrast(image).astype(np.uint8)

                        # Normalize and segment the image
                        img_norm = normalize(image, 1, 99.8, axis=(0, 1))
                        labels, details = model.predict_instances(img_norm)

                        # Save the images
                        imsave(os.path.join(output_folder, f'{base_name}_original_{j}.tif'), image)
                        imsave(os.path.join(output_folder, f'{base_name}_segmented_{j}.tif'), labels)
                        merged_image = np.dstack((image, labels, np.zeros_like(image)))
                        imsave(os.path.join(output_folder, f'{base_name}_merged_{j}.tif'), merged_image)

                        # Create figure for the PDF
                        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
                        axes[0].imshow(image, cmap='gray')
                        axes[0].set_title('Original')
                        axes[1].imshow(labels, cmap='magma')
                        axes[1].set_title('Segmented')
                        axes[2].imshow(merged_image)
                        axes[2].set_title('Merged')

                        # Annotate with the original TIF filename
                        fig.suptitle(base_name, fontsize=10)

                        # Adjust layout to avoid overlapping
                        fig.tight_layout(rect=[0, 0, 1, 0.95])

                        # Save the current figure into the PDF
                        pdf.savefig(fig)
                        plt.close(fig)  # Close the figure to free up memory

                        # Clear variables to free up memory
                        del image, labels, merged_image, img_norm

            except Exception as e:
                print(f"Error processing image from file {image_file}: {e}")

    print(f"PDF report saved to: {pdf_path}")

# Process images and generate PDF report in the correct order
process_images(rgb_image_files, grayscale_tiff_files, upsampled_tiff_files, output_dir)


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.7 MB/s eta 0:00:00
Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 759 (delta 0), reused 2 (delta 0), pack-reused 755
Receiving objects: 100% (759/759), 5.39 MiB | 3.66 MiB/s, done.
Resolving deltas: 100% (406/406), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00041_CH3_gray_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_gray_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00006_CH3_gray_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00037_CH3_gray_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00025_CH3_gray_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00005_CH3_gray_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (148): /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00036_CH3_gray_1.tif is a low contrast image


/content/Real-ESRGAN
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 334MB/s]
Testing 0 Muc1_Ecad_SPC_x20_2_XY10_00001_CH3_1
Testing 1 Muc1_Ecad_SPC_x20_2_XY10_00002_CH3_1
Testing 2 Muc1_Ecad_SPC_x20_2_XY10_00003_CH3_1
Testing 3 Muc1_Ecad_SPC_x20_2_XY10_00004_CH3_1
Testing 4 Muc1_Ecad_SPC_x20_2_XY10_00005_CH3_1
Testing 5 Muc1_Ecad_SPC_x20_2_XY10_00006_CH3_1
Testing 6 Muc1_Ecad_SPC_x20_2_XY10_00007_CH3_1
Testing 7 Muc1_Ecad_SPC_x20_2_XY10_00008_CH3_1
Testing 8 Muc1_Ecad_SPC_x20_2_XY10_00009_CH3_1
Testing 9 Muc1_Ecad_SPC_x20_2_XY10_00010_CH3_1
Testing 10 Muc1_Ecad_SPC_x20_2_XY10_00011_CH3_1
Testing 11 Muc1_Ecad_SPC_x20_2_XY10_00012_CH3_1
Testing 12 Muc1_Ecad_SPC_x20_2_XY10_00013_CH3_1
Testing 13 Muc1_Ecad_SPC_x20_2_XY10_00014_CH3_1
Testing 14 Muc1_Ecad_SPC_x20_2_XY10_00015_CH3_1
Testing 15 Muc1_Ecad_SPC_x20_2_XY10_00016_CH3_1
Testing 16 Muc1_Ecad_SPC_

<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00005_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00006_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00025_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00031_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00035_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00036_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00037_CH3_1_out.tif is a low contrast image
<ipython-input-1-a2323bf97049> (79): /con

Processing file: /content/sample_data/mono/Muc1_Ecad_SPC_x20_2_XY10_00001_CH3.tif
Processing image from file /content/sample_data/mono/Muc1_Ecad_SPC_x20_2_XY10_00001_CH3.tif
Image shape: (720, 960), Image dtype: uint8


...

Processing file: /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_gray_1.tif
Processing image from file /content/output/grayscale_tiff/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_gray_1.tif
Image shape: (120, 160), Image dtype: uint8


<ipython-input-1-a2323bf97049> (209): /content/output/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_segmented_1.tif is a low contrast image
<ipython-input-1-a2323bf97049> (211): /content/output/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_merged_1.tif is a low contrast image


Processing file: /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_1_out.tif
Processing image from file /content/output/upsampled_tiff/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_1_out.tif
Image shape: (720, 960, 3), Image dtype: uint8


<ipython-input-1-a2323bf97049> (208): /content/output/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_original_2.tif is a low contrast image
<ipython-input-1-a2323bf97049> (209): /content/output/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_segmented_2.tif is a low contrast image
<ipython-input-1-a2323bf97049> (211): /content/output/Muc1_Ecad_SPC_x20_2_XY10_00042_CH3_merged_2.tif is a low contrast image


PDF report saved to: /content/output/report.pdf
